<a href="https://colab.research.google.com/github/mrugeles/covid19-col/blob/formularios/Modelo_Regresion_Covid_19_Colombia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modelo de regresión sobre casos en Colombia de Covid-19

Este es un ejemplo de un modelo de regresión con un dataset muy limitado de apenas 14 registros (Marzo 19 del 2020) disponibles en la página del [Instituto Nacional de Salud](https://www.ins.gov.co/Noticias/Paginas/Coronavirus.aspx).

Este notebook se ha creado con fines educativos para demostrar cómo realizar un modelo de regresión polinomial.

Hay que tener en cuenta que la evolución de la infermedad con el tiempo define una curva logística y no una exponencial, esto es, la curva alcanza un punto de inflexión para luego tener una tendencia hacia una asintota horizonal.

In [1]:
import pandas as pd
import numpy as np
import operator
import datetime

import plotly.graph_objects as go

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures

'''
Dataset disponible en:

https://e.infogram.com/3a4c9dd3-fbb8-4f59-8d82-eb1edc01604b?parent_url=https%3A%2F%2Fwww.ins.gov.co%2FNoticias%2FPaginas%2FCoronavirus.aspx&src=embed#
'''
dataset = pd.read_csv('/content/drive/My Drive/Casos1.csv')
dataset['Fecha de diagnóstico'] = pd.to_datetime(dataset['Fecha de diagnóstico'])
dataset.head()

,ID de caso,Fecha de diagnóstico,Ciudad de ubicación,Atención,Edad,Sexo,Tipo,País de procedencia
0,1,2020-03-06,Bogotá,casa,10 a 19,F,Importado,Italia
1,2,2020-03-09,Buga,hospital,30 a 39,M,Importado,España
2,3,2020-03-09,Medellín,casa,50 a 59,F,Importado,España
3,4,2020-03-11,Medellín,casa,50 a 59,M,Relacionado,Colombia
4,5,2020-03-11,Medellín,casa,20 a 29,M,Relacionado,Colombia


In [2]:


fig = go.Figure()

grouped_df = dataset.groupby(pd.Grouper(key='Fecha de diagnóstico', freq='1d')).count().reset_index().rename(columns={'ID de caso':'count'})
fig.add_trace(go.Bar(x=grouped_df['Fecha de diagnóstico'], y=grouped_df['count']))


fig.update_layout(
    xaxis_tickformat = '%Y-%m-%d',
    xaxis_tickangle = -45,
    title_text="Contagios por día"
)
fig.show()

In [3]:
fig = go.Figure()
grouped_df['sum'] = grouped_df['count'].cumsum()
fig.add_trace(go.Bar(x=grouped_df['Fecha de diagnóstico'], y=grouped_df['sum']))


fig.update_layout(
    xaxis_tickformat = '%Y-%m-%d',
    xaxis_tickangle = -45,
    title_text="Acumulado de contagios por día"
)
fig.show()

In [4]:
df = grouped_df[['sum']].reset_index().rename(columns={'index':'x'})
fig = go.Figure(data=go.Scatter(x=df['x'], y=df['sum']))
fig.show()

In [0]:
days = 100

x = df['x'].values
y = df['sum'].values

x = x[:, np.newaxis]
y = y[:, np.newaxis]

polynomial_features= PolynomialFeatures(degree=3)
x_poly = polynomial_features.fit_transform(x)

x_predic = np.arange(0, days)
x_predic = x_predic[:, np.newaxis]
x_predic = polynomial_features.fit_transform(x_predic)


model = LinearRegression()
model.fit(x_poly, y)
y_predic = model.predict(x_predic).flatten()


In [6]:
predictions = pd.DataFrame(columns=['x', 'y'])
predictions['x'] = x_predic[:, 1]
predictions['y'] = y_predic
predictions.head()

,x,y
0,0.0,1.000000
1,1.0,1.167582
2,2.0,1.428571
3,3.0,2.197303
4,4.0,3.888112


In [0]:
predictions['date'] = pd.date_range(start='3/6/2020', periods=days)
grouped_df['date'] = pd.date_range(start='3/6/2020', periods=grouped_df.shape[0])

In [8]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=grouped_df['date'], y=grouped_df['sum'], mode='lines+markers', name='Predicción'))
fig.add_trace(go.Scatter(x=predictions['date'], y=predictions['y'], mode='lines', name='Predicción'))

fig.update_layout(
    xaxis_tickformat = '%Y-%m-%d',
    xaxis_tickangle = -45,
    title_text="Predicción para el mes de marzo"
)
fig.show()